In [19]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
import sys
import os
sys.path.append(os.path.abspath(".."))

from functools import reduce
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sparseCodedLayer import SparseCodedLayer

In [ ]:
def xor(x):
    return reduce(lambda a, b: a ^ b, x)

batch_size = 5000
in_features = 5
X = np.random.randint(0, 2, size=(batch_size, in_features))
y = np.array([xor(x) for x in X])

X = torch.tensor(X).float()
y = torch.tensor(y).long()

In [85]:
X_train, y_train = X[:4000], y[:4000]
X_val, y_val = X[4000:], y[4000:]

In [ ]:
class SimpleSparseModel(nn.Module):
    def __init__(self, in_features = in_features, out_features = 2):
        super().__init__()
        self._layer1 = SparseCodedLayer(in_features, 10, out_features_allowed=5, activation=nn.LeakyReLU(0.1))
        self._layer2 = SparseCodedLayer(10, 15, out_features_allowed=5,activation=nn.LeakyReLU(0.1))
        self._layer3 = SparseCodedLayer(15, out_features, activation=nn.LeakyReLU(0.1))
        nn.Linear(in_features, out_features).forward
        self.net = nn.Sequential(
            self._layer1,
            self._layer2,
            self._layer3
        )

    def forward(self, x):
        return self.net(x)

In [96]:
model = SimpleSparseModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
for epoch in range(300):
    optimizer.zero_grad()
    outputs = model(X_train)
    
    loss = criterion(outputs, y_train)

    loss.backward()
    optimizer.step()

    if (epoch+1) % 50 == 0:
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

Epoch 50, Loss: 0.6807
Epoch 100, Loss: 0.5988
Epoch 150, Loss: 0.4354
Epoch 200, Loss: 0.3861
Epoch 250, Loss: 0.2828
Epoch 300, Loss: 0.1446


In [97]:
op = model(X_val)
accuracy = (torch.argmax(op, dim=1) == y_val).float().mean().item()
print(f"Training accuracy: {accuracy*100:.2f}%")

Training accuracy: 96.80%
